In [11]:
import os, math, numpy as np

In [119]:
def get_travel_time(source: tuple, destination : tuple) -> float:
    try:
        return math.dist(source, destination) / 427
    except:
        print(source, destination)

In [120]:
### Calculate travel time between two points: origin and destination
def traveltime(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

# Read in data instance
A folder of data instance has 5 files: (1) couriers.txt, (2) instance_characteristics.txt, (3) instance_parameters.txt, (4) orders.txt, (5) restaurants.txt

In [121]:
# Read instance information
def read_instance_information(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])
    # locations.set_index('id',inplace=True)
    #
    # orders.set_index('order',inplace=True)
    # couriers.set_index('courier',inplace=True)
    # restaurants.set_index('restaurant',inplace=True)

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']

    return orders,restaurants,couriers,instanceparams,locations

In [122]:
import pandas as pd
instance_dir = 'data/0o50t75s1p100'
orders,restaurants,couriers,instanceparams,locations=read_instance_information(instance_dir)

# Set hyper-parameters:
f∶ every f minutes solves a matching problem to prescribe the next pick-up and delivery assignment for each courier<br>
t : optimization time<br>
∆_(u): t+ ∆_(u ) is the assignment horizon<br>
∆_(1); ∆_(2): to determine Z_(t)<br>
beta: control the freshness in the construction of bundles
gamma: control the click to door time in the construction of bundles


In [123]:
# Accorting to default values in the paper
f = 5
delta_u = 10
delta_1 = 10
delta_2 = 10
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned

In [124]:
restaurant_dict = restaurants.set_index('restaurant').to_dict()

# Procedure to initialize a solution

**Function that return U(t,r)**<br>
U(t,r) is the set of upcoming orders at restaurant r

**Courier Object**

In [262]:
class Order:
    def __init__(self, order_information : dict):

        self.order_id = order_information.get('order')
        self.destination = (order_information.get('x'), order_information.get('y'))
        self.placement_time = order_information.get('placement_time')
        self.restaurant_id = order_information.get('restaurant')
        self.ready_time = order_information.get('ready_time')

        self.restaurant_coord = (restaurant_dict['x'][self.restaurant_id], restaurant_dict['y'][self.restaurant_id])
        self.delivered_time = None

    def __repr__(self):
        return "order_id : {} destination : {} placement_time : {} restaurant_id : {} ready_time : {} delivered_time : ".format(
            self.order_id, self.destination, self.placement_time, self.restaurant_id, self.ready_time, self.delivered_time
        )


In [263]:
class Courier(object):
    def __init__(self, courier_information : dict, restaurant : dict):
        self.courier = courier_information.get('courier')
        self.on_time = courier_information.get('on_time')
        self.off_time = courier_information.get('off_time')

        # Update and derived along the way
        self.travel = []
        self.visited = []

        # self.next_available_time = # a dictionary containing restaurant as key and available time as value
        self.final_position = self.interim_position = (courier_information.get('x'), courier_information.get('y'))
        self.distance_travelled = 0

        self.final_time = self.interim_time = self.on_time

    @staticmethod
    def calculate_travel_time(source: tuple,
                              restaurant:tuple,
                              destination: tuple) -> float:

        return get_travel_time(source, restaurant) + get_travel_time(restaurant, destination)

   # check if the courier can take the order
    def can_assign(self, order : Order) -> bool:
        # if the ready time < courier's off time
        if order.ready_time < self.on_time:
            return False
        if order.ready_time > self.off_time:
            return False

    # assign an order to courier
    def assign_order(self, order: Order) -> bool:
        # update available time for the next assignment
        _, restaurant_x, restaurant_y = restaurants[restaurants['restaurant'] == order.restaurant].values[0]
        if self.can_assign(order):
            self.orders.append(order)
            self.interim_time += self.calculate_travel_time(self.interim_position, (restaurant_x, restaurant_y), orders.destination)

    def __repr__(self):
        return "courier_id : {} current_coord : {} on_time : {} off_time : {} current_time : {}".format(
            self.courier, self.final_position, self.on_time, self.off_time, self.final_time
        )

In [404]:
class DeliveryRouting:
    def __init__(self, instance_dir : str, target_bundle_size : int = 5, f : int = 5, delta_u : int = 10):
        self.target_bundle_size = target_bundle_size
        self.delta_u = delta_u
        self.f = f

        orders, restaurants, couriers, instanceparams, locations = read_instance_information(instance_dir)

        self.orders = [Order(order) for order in orders.to_dict(orient = 'records')]
        self.orders = sorted(self.orders, key = lambda x: x.ready_time)

        self.restaurants = restaurants.to_dict(orient = 'records')
        self.couriers = [Courier(courier, self.restaurants) for courier in couriers.to_dict(orient = 'records')]
        self.unassigned_orders = self.orders.copy()

        self.instance_params = instanceparams.to_dict(orient = 'records')[0]

        self.meters_per_minute, self.pickup_service_minutes, self.dropoff_service_minutes, \
            self.target_click_to_door, self.maximum_click_to_door, self.pay_per_order,\
            self.guaranteed_pay_per_hour = self.instance_params.values()

        self.bundles = {}
        self.bundles_assigned = {}

    @staticmethod
    def calculate_travel_time(source: tuple,
                              restaurant:tuple,
                              destination: tuple = None) -> float:

        if destination is not None:
            return get_travel_time(source, restaurant) + get_travel_time(restaurant, destination)
        return get_travel_time(source, restaurant)

    def check_can_bundle(self,
                         courier : Courier,
                         existing_bundle : [Order],
                         new_order : Order):

        total_time_taken = 0
        max_leave_time = max([order.ready_time for order in existing_bundle])
        max_leave_time = max(self.calculate_travel_time(courier.final_position, new_order.restaurant_coord), max_leave_time)
        max_leave_time = max(max_leave_time, new_order.ready_time) + 4

        if max_leave_time >= existing_bundle[0].ready_time + 90:
            return False

        if existing_bundle[-1].restaurant_id != new_order.restaurant_id:
            return False

        if new_order.ready_time > courier.off_time:
            return False

        if new_order.ready_time < courier.on_time:
            return False

        return max_leave_time

    def get_bundle_size(self, z : int) -> int :
        # available = [courier for courier in self.couriers if courier.]
        pass

    def __is_unassigned(self,
                      order_id : str) -> bool:

        for order in self.unassigned_orders:
            if order.order_id == order_id:
                return True
        return False

    def __assign_order(self,
                       courier : Courier,
                       to_assign_order : Order) -> bool:

        courier.current_time = self.calculate_travel_time(courier.interim_position, to_assign_order.restaurant_coord, to_assign_order.destination)
        courier.interim_position = to_assign_order.destination

        for order in self.unassigned_orders:
            if order.order_id == to_assign_order.order_id:
                self.unassigned_orders.remove(order)
                return

    def __get_available_couriers(self,
                                 t : int):
        return [courier for courier in self.couriers if courier.on_time <= t and courier.off_time >= t]

    def get_available_couriers(self,
                                 t : int):
        return [courier for courier in self.couriers if courier.on_time <= t and courier.off_time >= t]

    def __get_available_orders(self,
                               t : int):
        return [order for order in self.unassigned_orders if order.ready_time <= t + self.delta_u]

    def get_available_orders(self,
                               t : int):
        return [order for order in self.unassigned_orders if order.ready_time <= t + self.delta_u]

    def __get_courier(self, courier_id : str):
        for courier in self.couriers:
            if courier.courier == courier_id:
                return courier

    def __update_order(self,
                       order_id,
                       delivered_time):

        assert isinstance(order_id, str), "Please use Order ID instead."

        arr = []
        for order in self.orders:
            if order.order_id == order_id:
                order.delivered_time = delivered_time
            arr.append(order)
        self.orders = arr

    def __update_courier(self,
                         courier_id,
                         time,
                         location):
        arr = []
        for courier in self.couriers:
            if courier.courier == courier_id:
                courier.final_time = courier.interim_time = time
                courier.final_location = courier.current_location = location
            arr.append(courier)
        self.couriers = arr

    def assign_task(self, t : int):
        available_orders = self.__get_available_orders(t)
        available_couriers = self.__get_available_couriers(t)

        for order in available_orders:

            # Check whether can bundle first
            for bundle_id, bundle in self.bundles.items():
                if self.__is_unassigned(order.order_id):
                    courier_assigned_id = self.bundles_assigned[bundle_id]
                    courier = self.__get_courier(courier_assigned_id)
                    if self.check_can_bundle(courier, bundle, order):
                        bundle.append(order)
                        self.__assign_order(courier, order)


            if self.__is_unassigned(order.order_id):

                for _, bundle in self.bundles.items():
                    courier_assigned_id = self.bundles_assigned[bundle_id]
                    courier = self.__get_courier(courier_assigned_id)
                    self.move_courier(courier.courier, bundle)


                min_time = 1e9
                is_assigned = False
                for courier in available_couriers:
                    courier_time = courier.final_time

                    travel_time = courier_time + self.calculate_travel_time(courier.final_position,order.restaurant_coord)
                    leave_time = max(travel_time, order.ready_time) + 4
                    next_leg = self.calculate_travel_time(order.restaurant_coord, order.destination) + 4

                    if (leave_time + next_leg) < min_time:
                        min_time = leave_time + next_leg
                        courier_assigned = courier
                        is_assigned = True

                if is_assigned:
                    bundle_id = len(self.bundles_assigned.keys())
                    self.bundles[bundle_id] = [order]
                    self.bundles_assigned[bundle_id] = courier_assigned.courier

    def move_courier(self, courier_id : int, list_of_orders_to_move : [Order]):
        courier = self.__get_courier(courier_id)
        bundle_assigned = list_of_orders_to_move

        for order in list_of_orders_to_move:
            self.__assign_order(courier, order)

        time = self.calculate_travel_time(courier.final_position, bundle_assigned[0].restaurant_coord) + 4

        if len(list_of_orders_to_move) > 1:
            for index in range(len(bundle_assigned) - 1):
                previous_order = bundle_assigned[index]
                restaurant_coord = previous_order.restaurant_coord

                next_order = bundle_assigned[index]

                if index == 0:
                    delivered_time = self.calculate_travel_time(courier.final_position, restaurant_coord, previous_order.destination) + 4
                    self.__update_order(previous_order.order_id, delivered_time)

                time += self.calculate_travel_time(restaurant_coord, previous_order.destination, next_order.destination) + 4
                self.__update_order(next_order.order_id, delivered_time)

        else:
            next_order = list_of_orders_to_move[0]
            time = self.calculate_travel_time(courier.final_position, bundle_assigned[0].restaurant_coord, bundle_assigned[0].destination) + 8
            self.__update_order(next_order.order_id, self.calculate_travel_time(courier.final_position, bundle_assigned[0].restaurant_coord, bundle_assigned[0].destination) + 4)


        self.__update_courier(courier_id, time, next_order.destination)

In [405]:
instance_dir = 'data/0o50t75s1p100'
dr = DeliveryRouting(instance_dir)
dr.assign_task(60)

In [406]:
dr.bundles

{0: [order_id : o146 destination : (3695, 3690) placement_time : 13 restaurant_id : r54 ready_time : 28 delivered_time : ],
 1: [order_id : o89 destination : (7663, 2842) placement_time : 24 restaurant_id : r50 ready_time : 29 delivered_time : ],
 2: [order_id : o240 destination : (9386, 5834) placement_time : 29 restaurant_id : r67 ready_time : 46 delivered_time : ,
  order_id : o159 destination : (10213, 5982) placement_time : 36 restaurant_id : r67 ready_time : 56 delivered_time : ],
 3: [order_id : o227 destination : (9334, 6307) placement_time : 33 restaurant_id : r39 ready_time : 48 delivered_time : ]}

In [407]:
dr.bundles_assigned

{0: 'c1', 1: 'c1', 2: 'c1', 3: 'c1'}

**Funtion that calculate bundel size Z(t)**

**Function that calculate k(r,t)**<br>
k(r,t) is the number of couriers who are available at restaurant r

In [353]:
def get_available_courier(t,r):
    ...

**Route object**<br>
A route is a bundle,i.e: a list of ordered orders

In [ ]:
class Route(object):
    def __init__:
    
    # calculate total travel time from 1st destination to the last destination of the route
    def total_travel_time():

    # calculate total service delay    
    def total_service_delay():
        #<to be discuss>

    # calculate total_click_to_door 
    def total_click_to_door():
        #<to be discuss>

    # calculate route efficiency: travel time per order:
    def route_efficiency():

    # calculate route cost
    def route_cost():
        #route_cost = total_travel_time + beta*total_service_delay + gamma*total_click_to_door


# Implement procedure 1